Shamelessly plagerized from Ed Platt's [WP Efficiency Performance](https://github.com/elplatt/WP-Efficiency-Performance) repo.  (With permission.)

In [1]:
import ConfigParser
import os.path
import sqlalchemy
#import schema  # This is a module you create w/ SQLAlchemy calls to create the DB tables

In [ ]:
config = ConfigParser.ConfigParser()
path = os.path.abspath(os.path.join(os.path.dirname(__file__),".."))
config_file = os.path.join(path, 'config')
config.read(config_file)

In [6]:
os.path.abspath(os.path.join(os.path.dirname('.'),".."))

'/Users/lruhlen/Projects/panopticon'

In [ ]:
engine = sqlalchemy.create_engine(
    config.get("database", "url")
    , echo=False
    , pool_size = 26)

def create_tables():
    schema.Base.metadata.create_all(engine)

Following this [StackOverflow post](https://stackoverflow.com/questions/6506578/how-to-create-a-new-database-using-sqlalchemy):

In [1]:
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database

engine = create_engine("postgres://localhost/mydb")
if not database_exists(engine.url):
    create_database(engine.url)

print(database_exists(engine.url))

True


From [this gist](https://gist.github.com/sprin/5846464):

In [2]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    Integer,
)
from sqlalchemy.pool import NullPool

# XXX: It is advised to use another user that can connect to a default database,
# and has CREATE DATABASE permissions, rather than use a superuser.
DB_CONFIG_DICT = {
        'user': 'postgres',
        'password': 'postgres',
        'host': 'localhost',
        'port': 5432,
}

DB_CONN_FORMAT = "postgresql://{user}:{password}@{host}:{port}/{database}"

DB_CONN_URI_DEFAULT = (DB_CONN_FORMAT.format(
    database='postgres',
    **DB_CONFIG_DICT))

engine_default = create_engine(DB_CONN_URI_DEFAULT)

NEW_DB_NAME = 'test'

DB_CONN_URI_NEW = (DB_CONN_FORMAT.format(
    database=NEW_DB_NAME,
    **DB_CONFIG_DICT))

metadata = MetaData()

proj = Table('test', metadata, Column('id', Integer))

def setup_module():
    conn = engine_default.connect()
    conn.execute("COMMIT")
    # Do not substitute user-supplied database names here.
    conn.execute("CREATE DATABASE %s" % NEW_DB_NAME)
    conn.close()

def test_create_table():
    # Get a new engine for the just-created database and create a table.
    engine_new = create_engine(DB_CONN_URI_NEW, poolclass=NullPool)
    conn = engine_new.connect()
    metadata.create_all(conn)
    conn.close()

def teardown_module():
    conn = engine_default.connect()
    conn.execute("COMMIT")
    # Do not substitute user-supplied database names here.
    conn.execute("DROP DATABASE %s" % NEW_DB_NAME)
    conn.close()

ImportError: No module named psycopg2